<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
def check_diag_dom(matriz):
    n=len(matriz)
    diago_dom=True
    for i in range(n):
        sum=0
        for j in range(n):
            sum+=matriz[i][j]
        if(2*matriz[i][i]<sum):
            diago_dom=False;
            break;
    return (diago_dom)

In [16]:
import math
def dist_rel(x,y):#x=(x_1,x_2,...x_n), y=(y_1,y_2,...y_n)
    numerador=0
    denominador=0
    for i in range(len(x)):
        numerador+=(x[i]-y[i])**2
        denominador+=(x[i])**2
    return(math.sqrt(numerador/denominador))

In [17]:
def Jacobi(matriz,b,**x_0,tol=10**(-6),max_iter=1000):
    if(!check_diag_dom(matriz)): 
        return("la matriz no es diagonalmente dominante")
    else:
        count=0
        n=len(matriz)
        if(x_0):
            x_1=x_0
        else:
            x_0=np.zeros(n)
            x_1=np.zeros(n)
        while(count<=max_iter):
            for i in range(n):
                sum=0
                for j in range(n):
                    if(j!=i):
                        sum+=matriz[i][j]*x_0[j]
                x_1[i]=(b[i]-sum)/matriz[i][i]
            if(dist_rel(x_1,x_0)<tol):
                break;
            x_0=x_1
        print("numero de iteraciones: ",count)
        return (x_1)

SyntaxError: ignored

sss


0

In [11]:
A=[[-0.1,1000,-1000],[-1000,0.1,1000],[-1000,1000,0.1]]
check_diag_dom(A)

False